# Building a RAG Database with BRAD

In [1]:
from BRAD import agent

2024-11-21 12:57:31.279295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 12:57:31.297192: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 12:57:31.302637: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 12:57:31.316573: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 12:57:38.951699: W tensorflow/comp

## Searching Arxiv

In [2]:
bot = agent.Agent(
        config='config-ex-1.json'
).chat()

Enter your Open AI API key:  ········



Would you like to use a database with BRAD [Y/N]?


 N


2024-11-21 12:57:59 INFO semantic_router.utils.logger local


Welcome to RAG! The chat log from this conversation will be saved to /home/jpic/BRAD-Examples/RAG-SCRAPE/output/November 21, 2024 at 12:57:40 PM/log.json. How can I help?


...

BRAD >> 2: 


2024-11-21 12:58:22,450 - INFO - Directory '/home/jpic/BRAD-Examples/RAG-SCRAPE/output/November 21, 2024 at 12:57:40 PM/pdf' created successfully
2024-11-21 12:58:28,757 - INFO - 


route



2024-11-21 12:58:28,759 - INFO - SCRAPE
2024-11-21 12:58:28,763 - INFO - current llm calls=1


The following articles were downloaded:

https://arxiv.org/pdf/2410.01853
https://arxiv.org/pdf/2410.15828
https://arxiv.org/pdf/2410.22729
https://arxiv.org/pdf/2410.18394
https://arxiv.org/pdf/2409.09548
https://arxiv.org/pdf/2409.19482
https://arxiv.org/pdf/2409.15080
https://arxiv.org/pdf/2407.13205
https://arxiv.org/pdf/2407.18181
https://arxiv.org/pdf/2407.00754



Input >>  exit


Thanks for chatting today! I hope to talk soon, and don't forget that a record of this conversation is available at: /home/jpic/BRAD-Examples/RAG-SCRAPE/output/November 21, 2024 at 12:57:40 PM/log.json


## RAG

### Create a Database

In [3]:
from BRAD import rag

In [4]:
rag_database = rag.create_database(
    docsPath='papers/',
    dbName='tutorialDatabase',
    dbPath='databases/',
    v=True
)

/home/jpic/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
2024-11-21 12:58:44,592 - INFO - Use pytorch device_name: cpu
2024-11-21 12:58:44,593 - INFO - Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5



Work Directory: /home/jpic/BRAD-Examples/RAG-SCRAPE


/home/jpic/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Documents loading from: 'str(docsPath)


100%|██████████| 1/1 [00:00<00:00,  5.10it/s]


Documents loaded...
Documents split into chunks...
Initializing Chroma Database...
dbName reset



2024-11-21 12:58:45,595 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-21 12:58:45,743 - INFO - Collection tutorialDatabase is not created.


In [5]:
bot = agent.Agent(
    ragvectordb=rag_database,
    config='config-ex-1.json'
)

2024-11-21 13:02:13 INFO semantic_router.utils.logger local


Welcome to RAG! The chat log from this conversation will be saved to /home/jpic/BRAD-Examples/RAG-SCRAPE/output/November 21, 2024 at 01:02:13 PM/log.json. How can I help?


In [6]:
bot.chat()

Input >>  what is cellular reprogramming?


2024-11-21 13:02:27,924 - INFO - RAG


BRAD >> 1: 


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Source: Algorithm for cellular reprogramming.pdf
Content: repro-
gramming. Nevertheless, it seems natural to treat reprogram-
ming as a problem in control theory, with the ﬁnal state being
the desired reprogrammed cell. In this paper, we provide such a
framework based on empirical data and demonstrate the poten-tial of this framework to provide insights into cellular repro-
gramming (4).
Our goal is to mathematically identify TFs that can directly
reprogram human ﬁbroblasts into a desired target cell type. As
part of our methodology, we create a model for the natural
dynamics of proliferating human ﬁbroblasts, using time series
data collected throughout the cell cycle. We couple data from

/home/jpic/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
2024-11-21 13:02:29,283 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-21 13:02:29,287 - INFO - 


route



2024-11-21 13:02:29,288 - INFO - RAG
2024-11-21 13:02:29,294 - INFO - current llm calls=1



> Finished chain.

> Finished chain.
Cellular reprogramming is the process of changing the identity or characteristics of a cell from one type to another. This can involve altering the gene expression patterns of a cell to convert it into a different cell type. It is often achieved through the introduction of specific transcription factors that can induce the desired changes in the cell's behavior and function.
Sources:
Algorithm for cellular reprogramming.pdf


Input >>  exit


Thanks for chatting today! I hope to talk soon, and don't forget that a record of this conversation is available at: /home/jpic/BRAD-Examples/RAG-SCRAPE/output/November 21, 2024 at 01:02:13 PM/log.json
